# Notebook used for data preprocessing. Using predefined functions, this notebook prepares texts of articles for ML tasks we perform.

Imports

In [ ]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification, pipeline
from lib.sentiment_analysis_utils import combine_lede_and_text, remove_text_formatting, read_all_news_in_dir
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
os.getcwd()

Read data

In [ ]:
df_en_raw = read_all_news_in_dir(os.getcwd() + "/../data_preparation/raw_data/en/")
df_en_raw

In [ ]:
df_en_raw = combine_lede_and_text(df_en_raw)
df_en_raw = remove_text_formatting(df_en_raw)
df_en_raw

Perform preprocessing using functions from lib. Lede and remaining text of article are firstly combined. Then text formatting is removed.

In [ ]:
df_en_raw = df_en_raw[df_en_raw['whole_text']!='']
df_en_raw = df_en_raw.drop_duplicates(subset=['whole_text'])

Drop duplicates and articles which text is empty.

In [ ]:
df_en_raw

In [ ]:
mlb = MultiLabelBinarizer()
df_en_raw = df_en_raw.join(pd.DataFrame(mlb.fit_transform(df_en_raw.categories),
                          columns=mlb.classes_,
                          index=df_en_raw.index))

We use MultiLabelBinarizer to extract categories into separate columns which will allow sampling.

For English, there are nine categories:
  - Advisory - AD
  - Arts and Culture - AC
  - Around Slovenia - AS
  - Business, finance and economy - BE
  - Health, environment, science - HE
  - Politics - PO
  - Roundup - RU
  - Schedule of Events - SE
  - Sports - ST

In [ ]:
df_en_raw

In [ ]:
authors = set()
for i, row in df_en_raw.iterrows():
    tmp = row.byline.split('/')
    for author in tmp:
        authors.add(author)

print(authors)

extracting authors

In [ ]:
tokenizer_ner = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model_ner = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

ner_pipeline = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner, grouped_entities=True, device=0)

we have decided to use Babelscape/wikineural-multilingual-ner model as it gave the most promising preliminary results. Mainly as it firstly allowed for grouped entities. (Entities with names composed of multiple words). Secondly, it was multilingual and trained on large corpora. Even though Slovenian was not among the languages it was trained on, having other than English languages helped. Even though articles are in English, naturally there are slovenian entities mentioned quite often. We believe that training on multiple languages still allowed for more accurate assessment when non-English entity was mentioned. (This model was the most sensitive returning on average the most entities, whereas other had tendency to omit some of the entities. + returned names were actually entities.)

In [ ]:
for i, row in df_en_raw.iterrows():
    ner_results = ner_pipeline(row.whole_text)
    ner_list = set([result['word'] for result in ner_results])

    df_en_raw.at[i, 'ner_list'] = ''
    df_en_raw.at[i, 'ner_list'] = list(ner_list)

    keywords_lower = [keyword.lower() for keyword in row.keywords]
    df_en_raw.at[i, 'keywords_lower'] = ''
    df_en_raw.at[i, 'keywords_lower'] = keywords_lower

Producing the lists of ners and embedding them. keywords are turned to lowercase.

In [ ]:
df_en_raw

In [ ]:
df_en_raw.to_csv('whole_dataset.csv')

Saving preprocessed data into single csv file. This can be used to perform document based and abstract based sentiment analysis.